# Physics Informed Neural Networks <br> Airfoil Aerodymanics

## PINN

In [ ]:
import os
from dotenv import load_dotenv
import sys

In [ ]:
load_dotenv()

lib_dir = os.environ.get("LOCAL_LIB_DIR")
sys.path.append(lib_dir)

data_dir_airfoil = os.environ.get("LOCAL_DATA_DIR_AIRFOIL")
model_dir_airfoil = os.path.join(data_dir_airfoil, "models")

In [ ]:
import torch
import yaml
from airfoil_pinn import AirfoilPINN
from naca4digit_airfoil import Naca4DigitAirfoil
import utils
import matplotlib.pyplot as plt

In [ ]:
# device = utils.get_device()
device = "cpu"
print(f"Device: {device}")

In [ ]:
with open("config_airfoil.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
rho = config["rho"]
mu = config["mu"]

in_velocity = config["in_velocity"]
out_pressure = config["out_pressure"]

# Domain limits
domain1 = utils.Domain2D(
        x_min=config["domain1"]["x_min"], 
        x_max=config["domain1"]["x_max"], 
        y_min=config["domain1"]["y_min"],
        y_max=config["domain1"]["y_max"])

domain2 = utils.Domain2D(
        x_min=config["domain2"]["x_min"], 
        x_max=config["domain2"]["x_max"], 
        y_min=config["domain2"]["y_min"],
        y_max=config["domain2"]["y_max"])

In [ ]:
# Define airfoil parameters (example: NACA 2412)
chord = 1.0
m = 0.02  # maximum camber
p = 0.4   # position of maximum camber
t = 0.12  # maximum thickness
num_points = 100
angle_of_attack = 10

airfoil = Naca4DigitAirfoil(chord, m, p, t, alpha_deg=angle_of_attack)

In [ ]:
hidden_units = config["hidden_units"]

if config["activation_function"] == "tanh":
    activation_function = torch.tanh
elif config["activation_function"] == "sigmoid":
    activation_function = torch.sigmoid
else:
    raise ValueError("Activation function not recognized")

pinn = AirfoilPINN(hidden_units, activation_function, airfoil).to(device)

In [ ]:
if config["optimizer"]["name"] == "adam":
    optimizer = torch.optim.Adam(pinn.parameters(), lr=config["optimizer"]["lr"])
elif config["optimizer"]["name"] == "lbfgs":
    optimizer = torch.optim.LBFGS(pinn.parameters(), lr=config["optimizer"]["lr"], line_search_fn="strong_wolfe")
else:
    raise ValueError("Optimizer not recognized")

In [ ]:
epochs = config["epochs"]
checkpoint_epochs = config["checkpoint_epochs"]

Nf = config["Nf"]   # num of collocation points -> pde evaluation
Nb = config["Nb"]   # num of points to evaluate boundary conditions
Ns = config["Ns"]   # num of points of the surface of the airfoil to evaluate boundary conditions
Nin = config["Nin"] # num of points inside the airfoil to evaluate boundary conditions

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
airfoil.plot(ax=ax)
domain1.plot(ax=ax)
domain2.plot(ax=ax, color="red")

plt.show()

In [ ]:
pinn.train_pinn(
  epochs=epochs, 
  optimizer=optimizer, 
  Nf1=Nf, 
  Nf2=Nf, 
  Nb=Nb, 
  Ns=Ns, 
  Nin=Nin,
  domain1=domain1,
  domain2=domain2,
  in_velocity=in_velocity, 
  out_pressure=out_pressure, 
  mu=mu, 
  rho=rho, 
  device=device, 
  checkpoint_epochs=checkpoint_epochs, 
  model_dir=model_dir_airfoil, 
)

In [ ]:
pinn.plot_learning_curves(output_dir=model_dir_airfoil)